# Documentation Review for Berlin Source Data

### Environment Setup:

#### Installing PostgreSQL Database Adapters

In [106]:
!pip install psycopg2-binary

### Workspace Setup:

#### Libraries and Dependency Initialization

In [108]:
# Import the necessary libraries
import psycopg2
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine, text 
from IPython.display import display  # Fixed syntax here
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

#### Establishing Connection to the 'berlin_source_data' Schema

In [109]:
import os
from sqlalchemy import create_engine

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME")

engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)


#### Data Discovery: Dynamic Metadata Extraction of Schema Tables

In [116]:
# Define query to fetch all physical table names from the specific Berlin schema
# I have filter for 'BASE TABLE' to exclude views or system tables

query_tables = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'berlin_source_data' 
  AND table_type = 'BASE TABLE';
"""
# Execute the query and load results into a DataFrame for easy manipulation

table_names_df = pd.read_sql_query(sql=text(query_tables), con=engine)

# Convert the table column to a list to iterate through during the review process

table_list = table_names_df['table_name'].tolist()

# Print summary for verification of the connection and schema content

print(f"Found {len(table_list)} tables: {sorted(table_list)}")


Found 44 tables: ['banks', 'bike_lanes', 'bus_stops', 'dental_offices', 'districts', 'doctors', 'emergency_stations', 'exhibition_centers', 'food_markets', 'galleries', 'government_offices', 'gyms', 'hospitals', 'kindergartens', 'libraries', 'long_term_listings', 'malls', 'milieuschutz_protection_zones', 'museums', 'neighborhoods', 'night_clubs', 'parking_spaces', 'parks', 'petstores', 'pharmacies', 'playgrounds', 'pools', 'post_offices', 'public_artworks', 'recycling_points', 'religious_institutions', 'sbahn', 'schools', 'schools_maximilian_burkhardt', 'short_term_listings', 'social_clubs_activities', 'spaetis', 'supermarkets', 'theaters', 'tram_stops', 'ubahn', 'universities', 'venues', 'veterinary_clinics']


#### Schema Inspection: Extracting Column Metadata and Key Constraints

In [114]:
# SQL Query to extract detailed metadata for a specific table
# It joins column definitions with key constraints to identify Primary/Foreign keys

query_columns = text("""
    SELECT DISTINCT
        cols.column_name, 
        cols.data_type,
        tc.constraint_type
    FROM information_schema.columns AS cols
    LEFT JOIN information_schema.key_column_usage AS kcu
        ON cols.column_name = kcu.column_name 
        AND cols.table_name = kcu.table_name
        AND cols.table_schema = kcu.table_schema  -- FIX: Match schema
    LEFT JOIN information_schema.table_constraints AS tc
        ON kcu.constraint_name = tc.constraint_name
        AND kcu.table_schema = tc.table_schema      -- FIX: Match schema
    WHERE cols.table_schema = 'berlin_source_data' 
      AND cols.table_name = :table_name
    ORDER BY cols.column_name;
""")


#### Schema Consolidation: Generating Structural Profiles for All Tables

In [124]:
#  Execute Loop Efficiently
tables_schema_dict = {}

# Use a connection context manager to ensure the connection is closed after the loop
with engine.connect() as conn:
    for table in table_list:

        # Execute parameterized query for the current table in the loop        
        df = pd.read_sql_query(query_columns, conn, params={"table_name": table})
        
  #     #Consolidation Logic:
        # Some columns may have multiple constraints (e.g., PRIMARY KEY and FOREIGN KEY).
        # We group by column name/type and join constraints into a single string 
        # to ensure each column is represented by exactly one row in our review.
        df = df.groupby(['column_name', 'data_type'])['constraint_type']\
            .apply(lambda x: ', '.join(filter(None, x))).reset_index()
        
        # Store the processed DataFrame in our master dictionary
        tables_schema_dict[table] = df

print(f"Successfully processed {len(tables_schema_dict)} tables.")



Successfully processed 44 tables.


#### Schema Validation: Targeted Quality Audit of each Tables

In [122]:
tables_schema_dict["dental_offices"]

,column_name,data_type,constraint_type
0,city,character varying,
1,district,character varying,
2,district_id,character varying,FOREIGN KEY
3,email,character varying,
4,geometry,character varying,
5,housenumber,character varying,
6,id,character varying,PRIMARY KEY
7,latitude,numeric,
8,longitude,numeric,
9,name,character varying,


In [54]:
tables_schema_dict["banks"]

,column_name,data_type,constraint_type
0,atm,character varying,
1,brand,character varying,
2,district,character varying,
3,district_id,character varying,FOREIGN KEY
4,geometry,character varying,
5,housenumber,character varying,
6,id,character varying,PRIMARY KEY
7,latitude,numeric,
8,longitude,numeric,
9,name,character varying,


In [56]:
tables_schema_dict["bus_stops"]

,column_name,data_type,constraint_type
0,district,character varying,
1,district_id,character varying,FOREIGN KEY
2,geometry,USER-DEFINED,
3,id,character varying,PRIMARY KEY
4,latitude,numeric,
5,longitude,numeric,
6,name,character varying,
7,neighborhood,character varying,
8,neighborhood_id,character varying,
9,shelter,character varying,


In [59]:
tables_schema_dict["dental_offices"]

,column_name,data_type,constraint_type
0,city,character varying,
1,district,character varying,
2,district_id,character varying,FOREIGN KEY
3,email,character varying,
4,geometry,character varying,
5,housenumber,character varying,
6,id,character varying,PRIMARY KEY
7,latitude,numeric,
8,longitude,numeric,
9,name,character varying,


In [60]:
tables_schema_dict["districts"]

,column_name,data_type,constraint_type
0,district,character varying,
1,district_id,character varying,UNIQUE
2,geometry,USER-DEFINED,
3,neighborhood,character varying,


In [62]:
tables_schema_dict["exhibition_centers"]

,column_name,data_type,constraint_type
0,building,character varying,
1,district,character varying,
2,district_id,character varying,FOREIGN KEY
3,geometry,character varying,
4,house_number,character varying,
5,id,character varying,PRIMARY KEY
6,latitude,numeric,
7,longitude,numeric,
8,name,character varying,
9,neighborhood,character varying,


In [63]:
tables_schema_dict["galleries"]

,column_name,data_type,constraint_type
0,district,character varying,
1,district_id,character varying,FOREIGN KEY
2,fee,character varying,
3,geometry,character varying,
4,house_number,character varying,
5,id,character varying,PRIMARY KEY
6,latitude,numeric,
7,longitude,numeric,
8,name,character varying,
9,neighborhood,character varying,


In [100]:
tables_schema_dict["hospitals"]

,column_name,data_type,constraint_type
0,amenity_tag,character varying,
1,city,character varying,
2,country,character varying,
3,district,character varying,
4,district_id,character varying,FOREIGN KEY
5,email,character varying,
6,emergency,character varying,
7,geometry,character varying,
8,healthcare_tag,character varying,
9,housenumber,character varying,


In [64]:
tables_schema_dict["kindergartens"]

,column_name,data_type,constraint_type
0,address,character varying,
1,capacity,integer,
2,district,character varying,
3,district_id,character varying,FOREIGN KEY
4,full_address,character varying,
5,geometry,character varying,
6,id,character varying,PRIMARY KEY
7,latitude,numeric,
8,longitude,numeric,
9,name,character varying,


In [66]:
tables_schema_dict["long_term_listings"]

,column_name,data_type,constraint_type
0,address,text,
1,city,character varying,
2,district,character varying,
3,district_id,character varying,FOREIGN KEY
4,first_tenant,character varying,
5,floor,double precision,
6,geometry,text,
7,house_number,character varying,
8,id,character varying,PRIMARY KEY
9,latitude,numeric,


In [67]:
tables_schema_dict["milieuschutz_protection_zones"]

,column_name,data_type,constraint_type
0,amendment_announced,date,
1,amendment_effective,date,
2,area_ha,numeric,
3,date_announced,date,
4,date_effective,date,
5,district,character varying,
6,district_id,character varying,FOREIGN KEY
7,geometry,USER-DEFINED,
8,id,character varying,PRIMARY KEY
9,latitude,numeric,


In [68]:
tables_schema_dict["museums"]

,column_name,data_type,constraint_type
0,building,character varying,
1,district,character varying,
2,district_id,character varying,FOREIGN KEY
3,fee,character varying,
4,geometry,character varying,
5,house_number,character varying,
6,id,character varying,PRIMARY KEY
7,latitude,numeric,
8,longitude,numeric,
9,museum_type,character varying,


In [69]:
tables_schema_dict["neighborhoods"]

,column_name,data_type,constraint_type
0,district,character varying,
1,district_id,character varying,FOREIGN KEY
2,geometry,USER-DEFINED,
3,neighborhood,character varying,
4,neighborhood_id,character varying,


In [70]:
tables_schema_dict["parks"]

,column_name,data_type,constraint_type
0,area_sq_m,double precision,
1,district,character varying,
2,district_id,character varying,FOREIGN KEY
3,full_address,character varying,
4,geometry,character varying,
5,id,character varying,PRIMARY KEY
6,latitude,numeric,
7,longitude,numeric,
8,name,character varying,
9,neighborhood,character varying,


In [71]:
tables_schema_dict["playgrounds"]

,column_name,data_type,constraint_type
0,area_sq_m,double precision,
1,district,character varying,
2,district_id,character varying,FOREIGN KEY
3,full_address,character varying,
4,geometry,character varying,
5,id,character varying,PRIMARY KEY
6,latitude,numeric,
7,longitude,numeric,
8,name,character varying,
9,neighborhood,character varying,


In [72]:
tables_schema_dict["pharmacies"]

,column_name,data_type,constraint_type
0,coordinates,character varying,
1,district,character varying,
2,district_id,character varying,FOREIGN KEY
3,geometry,character varying,
4,housenumber,character varying,
5,id,character varying,PRIMARY KEY
6,latitude,numeric,
7,longitude,numeric,
8,name,character varying,
9,neighborhood,character varying,


In [73]:
tables_schema_dict["pools"]

,column_name,data_type,constraint_type
0,district,character varying,
1,district_id,character varying,FOREIGN KEY
2,geometry,character varying,
3,id,character varying,PRIMARY KEY
4,latitude,numeric,
5,longitude,numeric,
6,name,character varying,
7,neighborhood,character varying,
8,neighborhood_id,character varying,
9,open_all_year,boolean,


In [74]:
tables_schema_dict["public_artworks"]

,column_name,data_type,constraint_type
0,artist_name,character varying,
1,artwork_type,character varying,
2,district,character varying,
3,district_id,character varying,FOREIGN KEY
4,geometry,character varying,
5,id,character varying,PRIMARY KEY
6,image,character varying,
7,latitude,numeric,
8,longitude,numeric,
9,material,character varying,


In [ ]:
# Target table does not exist in the current schema
tables_schema_dict["regional_statistics"]

KeyError: 'regional_statistics'

In [ ]:
# Target table does not exist in the current schema
tables_schema_dict["rent_stats_per_neighborhood"]

KeyError: 'rent_stats_per_neighborhood'

In [77]:
tables_schema_dict["schools"]

,column_name,data_type,constraint_type
0,bsn,character varying,
1,district,character varying,
2,district_id,character varying,FOREIGN KEY
3,email,character varying,
4,fax_number,character varying,
5,geometry,character varying,
6,id,character varying,PRIMARY KEY
7,latitude,numeric,
8,longitude,numeric,
9,name,character varying,


In [78]:
tables_schema_dict["short_term_listings"]

,column_name,data_type,constraint_type
0,accommodates,integer,
1,amenities,text,
2,bathrooms,numeric,
3,bedrooms,numeric,
4,beds,numeric,
5,district,character varying,
6,district_id,character varying,FOREIGN KEY
7,geometry,USER-DEFINED,
8,host_id,bigint,
9,id,character varying,PRIMARY KEY


In [79]:
tables_schema_dict["ubahn"]

,column_name,data_type,constraint_type
0,district,character varying,
1,district_id,character varying,FOREIGN KEY
2,geometry,character varying,
3,id,character varying,PRIMARY KEY
4,latitude,numeric,
5,line,character varying,
6,longitude,numeric,
7,name,character varying,
8,neighborhood,character varying,
9,neighborhood_id,character varying,


In [80]:
tables_schema_dict["universities"]

,column_name,data_type,constraint_type
0,district,character varying,
1,district_id,character varying,FOREIGN KEY
2,enrollment,integer,
3,founded,integer,
4,geometry,character varying,
5,id,character varying,PRIMARY KEY
6,latitude,numeric,
7,longitude,numeric,
8,name,character varying,UNIQUE
9,neighborhood,character varying,


In [81]:
tables_schema_dict["venues"]

,column_name,data_type,constraint_type
0,address,character varying,
1,category,character varying,
2,cuisine,character varying,
3,district,character varying,
4,district_id,character varying,FOREIGN KEY
5,geometry,character varying,
6,id,character varying,PRIMARY KEY
7,latitude,numeric,
8,longitude,numeric,
9,name,character varying,


In [83]:
tables_schema_dict["social_clubs_activities"]

,column_name,data_type,constraint_type
0,amenity,character varying,
1,club,character varying,
2,district,character varying,
3,district_id,character varying,FOREIGN KEY
4,email,character varying,
5,full_address,text,
6,geometry,text,
7,housenumber,character varying,
8,id,character varying,PRIMARY KEY
9,latitude,numeric,


In [84]:
tables_schema_dict["post_offices"]

,column_name,data_type,constraint_type
0,city,character varying,
1,closure_periods,character varying,
2,district,character varying,
3,district_id,character varying,FOREIGN KEY
4,geometry,character varying,
5,house_no,character varying,
6,id,character varying,PRIMARY KEY
7,latitude,numeric,
8,location_type,character varying,
9,longitude,numeric,


In [85]:
tables_schema_dict["government_offices"]

,column_name,data_type,constraint_type
0,address,text,
1,city,character varying,
2,coordinate_type,character varying,
3,created_at,timestamp without time zone,
4,district,character varying,
5,district_id,character varying,FOREIGN KEY
6,email,character varying,
7,geometry,USER-DEFINED,
8,id,character varying,PRIMARY KEY
9,latitude,numeric,


In [86]:
tables_schema_dict["libraries"]

,column_name,data_type,constraint_type
0,amenity,character varying,
1,city,character varying,
2,country,character varying,
3,district,character varying,
4,district_id,character varying,FOREIGN KEY
5,final_email,character varying,
6,final_phone,character varying,
7,geometry,USER-DEFINED,
8,housenumber,character varying,
9,id,character varying,PRIMARY KEY


In [87]:
tables_schema_dict["religious_institutions"]

,column_name,data_type,constraint_type
0,address,character varying,
1,denomination_std,character varying,
2,district,character varying,
3,district_id,character varying,FOREIGN KEY
4,email,character varying,
5,geometry,character varying,
6,heritage,character varying,
7,historic,character varying,
8,housenumber,character varying,
9,id,character varying,PRIMARY KEY


In [88]:
tables_schema_dict["petstores"]

,column_name,data_type,constraint_type
0,brand,character varying,
1,district,character varying,
2,district_id,character varying,FOREIGN KEY
3,full_address,character varying,
4,id,character varying,PRIMARY KEY
5,latitude,double precision,
6,longitude,double precision,
7,name,character varying,
8,neighborhood,character varying,
9,neighborhood_id,character varying,


In [89]:
tables_schema_dict["tram_stops"]

,column_name,data_type,constraint_type
0,district,character varying,
1,district_id,character varying,FOREIGN KEY
2,geometry,USER-DEFINED,
3,id,character varying,PRIMARY KEY
4,latitude,numeric,
5,longitude,numeric,
6,name,character varying,
7,neighborhood,character varying,
8,neighborhood_id,character varying,
9,shelter,character varying,
